In [1]:
import networkx as nx
import collections 
import numpy as np
from scipy.stats import cauchy
from sklearn.isotonic import IsotonicRegression  
from sklearn.linear_model import LinearRegression 
import matplotlib
import matplotlib.pyplot as plt
import cvxpy as cp

In [2]:
#utility functions
def getSortedDegSeq(G):
    degSeq = sorted([d for n, d in G.degree()], reverse=False) #small to large degrees
    return degSeq

def getDegHis(G,maxDeg):
    degSeq = getSortedDegSeq(G)
    degreeCount = collections.Counter(degSeq)
    degHis = np.zeros(maxDeg+1)
    for deg in degreeCount:
        degHis[deg]=degreeCount[deg]    
    return degHis

def degSeqToDegHis(degSeq, maxDeg):
    #assume deg sequence could be non-integer and be bigger than maxDegree
    degHis = np.zeros(maxDeg+1)
    for deg in degSeq:
        #print(deg)
        deg = int(round(deg))
        if deg <= maxDeg:
            degHis[deg]= degHis[deg]+1
    return degHis
    
    
def pdfToCdf(pdf):
    cdf = np.zeros(len(pdf))
    cdf[0] = pdf[0]
    for i in range(1,len(pdf)):
         cdf[i] = cdf[i-1] + pdf[i]            
    return cdf

def cdfToPdf(cdf):
    pdf = np.zeros(len(cdf))
    pdf[0] = cdf[0]
    for i in range(1,len(pdf)):
         pdf[i] = cdf[i] - cdf[i-1]            
    return pdf
    

def difDegHis_L1(his1,his2):
    #assume the same length
    return sum(abs(his1 - his2))

def difDegHis_L2(his1,his2):
    return sum(np.square(his1-his2))



def plotHis(trueHis,noisyHis):
    plt.plot(trueHis,'-g', label='trueHis')
    plt.plot(noisyHis,'--r', label='noisyHis')
    plt.legend();
    plt.xscale('log')

    
def plotCum(trueHis,noisyHis):
    plt.plot(pdfToCdf(trueHis), '3b', label='trueCum')
    plt.plot(pdfToCdf(noisyHis), '2y', label='noisyCum')
    plt.legend();
    plt.xscale('log')

    
#DP basic functions
def lap(trueCounts, sens, epsilon):
    scale = 1.0* sens/epsilon
    noisyCounts = trueCounts + np.random.laplace(0.0, scale, len(trueCounts))
    return noisyCounts

def postprocessCdf(noisyCdf, totalCount):
    #apply isotonic regression
    ir = IsotonicRegression(y_min=0, y_max=totalCount, increasing=True)
    cdf= ir.fit_transform(X=range(len(noisyCdf)),y=noisyCdf)   
    return cdf

def postprocessPdf(noisyPdf, nodesNum):
    cdf = pdfToCdf(noisyPdf)
    cdf = postprocessCdf(cdf, nodesNum)
    pdf = cdfToPdf(cdf)
    return pdf


def extendHis(his,maxDeg):
    #his has a shortern length 
    if (maxDeg+1) > len(his):
        hisExtended = np.zeros(maxDeg + 1)
        hisExtended[0:len(his)] = his
        return hisExtended
    else:
        return his

    
def sampleProbList(probList):
    #print(probList)
    normalizedProbList = probList/sum(probList)
    #print(normalizedProbList)
    r = np.random.uniform(0,1,1)
    s = 0 
    for i in range(len(probList)):
        s += normalizedProbList[i]
        if s >= r:
            return i
    return len(probList)-1


#graph transformation/clean up for subgraph counting aglo (e.g. ladder function) 
#this remap the node id, such that node id starts from 0 and increments to the total number of nodes 
def translate(datafile, newDatafile):
    nodeMap = dict()
    
    fin = open(datafile, "r")
    fout = open(newDatafile, "w")
    for ij in fin:
        i,j = ij.split()
        #i = int(i)
        #j = int(j)
        if i not in nodeMap:
            nodeMap[i] = len(nodeMap)
        if j not in nodeMap:
            nodeMap[j] = len(nodeMap)
        
        i_map = nodeMap[i]
        j_map = nodeMap[j]
        if i_map < j_map:
            fout.write(str(i_map)+" "+str(j_map)+"\n")
        else:
            fout.write(str(j_map)+" "+str(i_map)+"\n")

    fout.close()
    fin.close()    


In [ ]:
#Edge DP algorithms for degree distribution

#Hay et al. ICDM'09 (baseline)
def edgeDP_degHis_Lap(G, maxDeg, epsilon):
    degHis = getDegHis(G, maxDeg)
    sens = 4.0
    noisyDegHis = lap(degHis, sens, epsilon)
    noisyDegHis = postprocessPdf(noisyDegHis, len(G.nodes()))
    return noisyDegHis


#Hay et al. ICDM'09, Proserpio et al. WOSN'12 (wPINQ)
def edgeDP_degSeq_Lap(G, maxDeg, epsilon):
    degSeq = np.array(getSortedDegSeq(G))
    sens = 2.0
    #print(degSeq)
    noisyDegSeq = lap(degSeq, sens,epsilon)
    noisyDegSeq = postprocessCdf(noisyDegSeq, maxDeg)
    #print(noisyDegSeq)
    noisyDegHis = degSeqToDegHis(noisyDegSeq, maxDeg)
    return noisyDegHis


In [ ]:
#Node DP algorithms for degree distribution
################

#Day et al. SIGMOD'16 
#Algo 1 (projection by edge-addition, variant: output degSeq instead of graph), part of Algo 4
def edgeAddition_DegList(G, theta):    
    #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    nodesNum = len(G.nodes())
    nodesList = list(G.nodes()) #the node ids are not strictly from 0 to |nodesNum|-1
    invNodesList = {}
    for id in range(nodesNum):
        v = nodesList[id]
        invNodesList[v]=id
    
    nodesIndices = np.random.permutation(nodesNum)
    degListGt = np.zeros(nodesNum)
    
    for vid in nodesIndices:
        v = nodesList[vid]
        for u in G.neighbors(v):
            uid = invNodesList[u]
            if uid<vid and degListGt[uid]<theta and degListGt[vid]<theta:
                degListGt[uid] = degListGt[uid]+1
                degListGt[vid] = degListGt[vid]+1
                
    degListGt=sorted(degListGt, reverse=False) #small to large degrees
    #print(degListGt)
    
    #degSeq = getSortedDegSeq(G)
    #diff = np.array(degSeq) - np.array(degListGt)
    #print("difference in deg seq after edge addition", sum(diff))
    
    return degListGt

#Day et al. SIGMOD'16 
#Part of Algo 4 (select an optimal theta) 
def learnTheta(G, maxDeg, epsilon_theta, epsilon_deg, thetaList):
    probList = []
    maxTheta = max(thetaList)
    sensitivity = 6.0 * maxTheta+4
    
    for theta in thetaList:
        degListGt = edgeAddition_DegList(G,theta)
        nodeNumTheta = len([deg for deg in degListGt if deg >theta])
        score = -2.0 * nodeNumTheta - np.sqrt(theta) * (theta+1)/epsilon_deg
        prob = np.exp(epsilon_theta * score /(2.0 * sensitivity))
        probList.append(prob)
        
    theta = thetaList[sampleProbList(probList)]
    return theta

#Day et al. SIGMOD'16 
#Part of Algo 2 (select an optimal theta and partition) 
def learnThetaPartition(G, maxDeg, epsilon_theta, epsilon_deg, thetaList, partitionList):
    probList = []
    maxTheta = max(thetaList)
    sensitivity = 6.0 * maxTheta+4
    
    his = getDegHis(G,maxDeg)
    for theta in thetaList:
        degListGt = edgeAddition_DegList(G,theta)
        nodeNumTheta = len([deg for deg in degListGt if deg >theta])
        lproj = 2.0 * nodeNumTheta
        for partition in partitionList:            
            lhist = 0 
            for i in range(len(partition)-1):
                start = partition[i]
                end = partition[i+1]
                ave = sum(his[start:end])/(end-start)
                diff = sum(abs(his[start:end] - ave))
                lhist = lhist + diff + (2.0*theta+1)/epsilon_deg
        score = -1.0 * (lhist + lproj)
        prob = np.exp(epsilon_theta * score /(2.0 * sensitivity))
        probList.append(prob)
        
    sampleIndex = sampleProbList(probList)
    thetaIndex = int(sampleIndex / len(partitionList))
    partitionIndex = sampleIndex % len(partitionList)
    theta= thetaList[thetaIndex]
    partition = partitionList[partitionIndex]
    return theta,partition


#Day et al. SIGMOD'16 
#Algo 3(extract histogram from cumulative histogram), part of Algo 4
def extractHist(cumHist): #cumHist is noisy cumulative histogram
    theta = len(cumHist)-1
    hist = np.zeros(len(cumHist))
    hc=list(cumHist)
    hc.append(cumHist[theta])
    hc[0] = 0
    i = 1
    if hc[1] <0:
        hc[1] = 0
    
    while i <= theta:
        if hc[i] <= hc[i+1]:
            hist[i] = max(hc[i] - hc[i-1],0)
            #print('a:', i, hist[i])
            i = i+1
        else:
            iold = i
            jlist = list(reversed(range(i,theta+2)))
            #print(jlist)
            for j in jlist:
                if hc[j-1] < hc[i]:
                    j = min(j,theta)
                    for k in range(i,j+1):
                        hist[k] = max(0,(hc[j]-hc[i-1])/(j-i+1))
                        #print('b:', k, hist[k])
                    i= j+1
                    break 
            if i == iold:
                #print("i does not change", i, cumHist[i:theta])
                break
    #print(pdfToCdf(hist))
    return hist
    

#Day et al. SIGMOD'16 
#Algo 5 (postprocess for tail destribution), part of Algo 4
#The last bin has a high count which contains the number of nodes with degree "at least" theta in G. 
def postTail(hist, theta):
    budget = hist[theta]
    #print("budget", budget)    
    start = int(round(theta/2))
    cbar = 2.0/theta * sum(hist[start:theta])
    
    X = np.array([[x] for x in range(start,theta)])
    #print(X)
    y = hist[start:theta]
    #print(y)
    reg = LinearRegression().fit(X,y)
    m = reg.coef_
    b = reg.intercept_ 
    #print(m,b)
    
    for k in range(theta, len(hist)):
        if m<0:
            #hist[k] = b + m * k #This step has an issue when theta is small, then b is small, and counts become negative
            hist[k] = max(b + m * k,0) 
        else:
            hist[k] = cbar
        budget = budget - hist[k]
        if budget <0:
            break
    return hist 


#Raskhodnikova & Smith, Arxiv'15
def flowgraph(G, theta):
    #https://www.cvxpy.org/examples/basic/quadratic_program.html
    #Raskhodnikova & Smith, Arxiv'15
    #Note: this algo is slow, take more than 1 min for nodeNum=200
    #May switch to a diffferent solver: https://pypi.org/project/qpsolvers/  (as @ is no longer recognized by new version of python)

    nodesNum = len(G.nodes()) 
    edgesNum = len(G.edges())
    print(nodesNum,edgesNum)

    nn = nodesNum * 2
    ne = edgesNum * 2
    n = nn + ne

    P = np.zeros([n,n])
    P[:(nn),:(nn)] = np.identity(nn)

    q = np.zeros(n)
    q[:(nn)] = -2.0* theta

    T = np.identity(n)
    h = np.zeros(n)+1
    h[:nn] = theta 

    #print(T,h)

    A = np.zeros([nn,n])
    edgeCount = nn
    for i in range(nodesNum):
        A[i,i] = 1 
        A[nodesNum+i,nodesNum+i] = 1
        neighborSize = len([k for k in G.neighbors(i)])
        for j in range(neighborSize):
            A[i,(edgeCount+j)] = -1
            A[nodesNum+i,(edgeCount+j)] = -1
        edgeCount = edgeCount + neighborSize
    #print(A)

    x = cp.Variable(n)
    prob = cp.Problem(cp.Minimize(cp.quad_form(x,P)+ q.T @ x), [T @x <=h, x>=0, A@x == 0 ])
    prob.solve()
    #print("\nThe optimal value is", prob.value)
    #print("A solution x is")

    degList = []
    for i in range(nodesNum):
        degList.append(int(x.value[i,0].round()))
    degSeq = sorted(degList,reverse=False)
    #print(degSeq)
    return degSeq

#Raskhodnikova & Smith, Arxiv'15
def flowgraphApprox(G,theta):
    #TODO: more efficient approximation algorithmm, but sensitivity will increase
    degSeq = []
    return degSeq

################
#Naive laplace mechanism with postprocessing
def nodeDP_degHis_Lap(G, maxDeg, epsilon):
    degHis = getDegHis(G,maxDeg)
    sens = 2.0 * (maxDeg + 1)
    noisyDegHis = lap(degHis, sens, epsilon)
    noisyDegHis = postprocessPdf(noisyDegHis, len(G.nodes()))
    return noisyDegHis

#Adapting Hay et al. ICDM'09, Proserpio et al. WOSN'12 (wPINQ)
def nodeDP_degSeq_Lap(G, maxDeg, epsilon):
    degSeq = np.array(getSortedDegSeq(G))
    sens = 1.0* (maxDeg + 1)
    #print(degSeq)
    noisyDegSeq = lap(degSeq, sens,epsilon)
    noisyDegSeq = postprocessCdf(noisyDegSeq, len(G.nodes()))
    #print(noisyDegSeq)
    noisyDegHis = degSeqToDegHis(noisyDegSeq, maxDeg)
    #print(noisyDegHis)
    return noisyDegHis

#Shiva et al. TCC'13
def nodeDP_nodeTrun_Smooth(G, maxDeg, epsilon, theta):
    epsilon_deg = epsilon 
    
    #node truncation: remove nodes with degree > theta 
    Gt = G.copy()
    nodesTrun = [n for n,d in Gt.degree() if d > theta]
    Gt.remove_nodes_from(nodesTrun)

    #smooth bound: (Prop 6.1, Algo 3)
    nodesNum = len(G.nodes())
    beta = epsilon/(np.sqrt(2.0)*(theta+1))   ## epsilon * np.log(nodesNum/theta)
    r = np.log(1.0*nodesNum/beta)
    l = len([n for n,d in Gt.degree() if (d >= theta-r and d<=theta+r)])
    smoothSens = l+1.0/beta + 1
    #print(beta, r, l,smoothSens)
    
    #add cauchy noise with epsilon_deg
    scale = 2*np.sqrt(2.0)*theta/epsilon_deg*smoothSens
    trueHisGt = getDegHis(Gt, theta)
    noisyHisGt = trueHisGt + cauchy.rvs(0,scale=smoothSens,size=len(trueHisGt))
    noisyHisG = extendHis(noisyHisGt, maxDeg)
    
    #postprocess (TCC DOES NOT HAVE THIS STEP)
    noisyHisG = postprocessPdf(noisyHisG, len(G.nodes()))
    
    return noisyHisG


#Raskhodnikova & Smith, Arxiv'15
def nodeDP_flowgraph_degSeq_Lap(G, maxDeg, epsilon, theta):
    epsilon_deg = epsilon 
    
    #TODO: add generalized exponential mechanism for selecting threshold theta
    
    
    #Flowgraph algorithm to obtain a degree sequence with max degree is bounded by theta
    degSeq = flowgraph(G,theta) #getSortedDegSeq(G) #TODO: replace this line with flowgraph algorithm 
    
    #Convert degSeq to degHis and add noise (this step can be improved by learn noisy seq first)
    sens = 6.0 *theta  #Algo 1: this requires an exact solver for the flowgraph; if an approximation algorithm is used, then the sensitivity needs go up.
    degHis = degSeqToDegHis(degSeq, theta)
    noisyDegHis = lap(degHis, sens, epsilon_deg)
    
    noisyHisG = extendHis(noisyDegHis, maxDeg)
    
    #postprocess (THE PAPER DOES NOT HAVE THIS STEP)
    noisyHisG = postprocessPdf(noisyHisG, len(G.nodes()))
    
    return noisyHisG


#Day et al. SIGMOD'16
def nodeDP_edgeAdd_degHisPart_Lap(G, maxDeg, epsilon, thetaList, paritionList):
    ##TODO: incomplete (exponential mechanism to get a partition first, and then apply Lap)
    return 0


#Day et al. SIGMOD'16
#variant of Algo 4 (\theta-constrained)
def nodeDP_edgeAdd_degCum_Lap_variant(G, maxDeg, epsilon, thetaList):
    theta = thetaList[0]
    epsilon_deg = epsilon
    
    #Learning theta 
    if len(thetaList) >1: #many choices 
        epsilon_theta = epsilon * 0.1
        epsilon_deg = epsilon-epsilon_theta
        theta = learnTheta(G, maxDeg, epsilon_theta, epsilon_deg, thetaList)
        #print("sampled theta", theta)

    #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    degListGt = edgeAddition_DegList(G,theta)
    #print(degListGt)
    
    #cumulative historm + lap noise
    degHis = degSeqToDegHis(degListGt, theta)
    degCum = pdfToCdf(degHis)
    #print(degCum[0:30]) 
    sens = theta + 1 
    noisyDegCum = lap(degCum, sens, epsilon_deg)
    
    #print("noisyDegCum", noisyDegCum[0:30])
    noisyDegCum = postprocessCdf(noisyDegCum, len(G.nodes())) #Not Algo 3
    #print("noisyDegCum - monotonicity", noisyDegCum[0:30])
    noisyDegHis = cdfToPdf(noisyDegCum)
    noisyDegHis = extendHis(noisyDegHis,maxDeg)
    #print("noisyDegHis - extend lengh", noisyDegHis)
    noisyDegHis = postTail(noisyDegHis, theta)
    #print("noisyDegHis - post tail", noisyDegHis)
    
    return noisyDegHis
        

#Day et al. SIGMOD'16
#Algo 4 (\theta-cumulative)
def nodeDP_edgeAdd_degCum_Lap(G, maxDeg, epsilon, thetaList):
    theta = thetaList[0]
    epsilon_deg = epsilon
    
    #Learning theta 
    if len(thetaList) >1: #many choices 
        epsilon_theta = epsilon * 0.1
        epsilon_deg = epsilon-epsilon_theta
        theta = learnTheta(G, maxDeg, epsilon_theta, epsilon_deg, thetaList)
        #print("sampled theta", theta)

    #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    degListGt = edgeAddition_DegList(G,theta)
    #print(degListGt)
    
    #cumulative historm + lap noise
    degHis = degSeqToDegHis(degListGt, theta)
    degCum = pdfToCdf(degHis)
    #print(degCum[0:30]) 
    sens = theta + 1 
    noisyDegCum = lap(degCum, sens, epsilon_deg)
    
    #print("noisyDegCum", noisyDegCum[0:30])
    noisyDegHis = extractHist(noisyDegCum) #Algo 3
    #print("noisyDegCum - monotonicity", noisyDegCum[0:30])
    
    noisyDegHis = extendHis(noisyDegHis,maxDeg)
    #print("noisyCumDegHis - extend lengh", noisyDegHis[0:30])
    noisyDegHis = postTail(noisyDegHis, theta)
    #print("noisyDegHis - post tail", noisyDegHis[0:30])
    
    return noisyDegHis


#Day et al. SIGMOD'16
#Algo 2 (\theta,\Omega-cumulative)
def nodeDP_edgeAdd_degHisPart_Lap(G, maxDeg, epsilon, thetaList, rList):
    theta = thetaList[0]
    epsilon_deg = epsilon

    #Create a set of partitions based on rList
    partitionList = []
    for r in rList:
        partition = [0]
        i = 0 
        k = int(round(np.power(r, i)))
        while k < maxDeg:
            if k > partition[len(partition)-1]:
                partition.append(k)
            i = i+1
            k = int(round(np.power(r, i)))
        partition.append(maxDeg) #we also add the maxDeg for the easy aggregation
        partitionList.append(partition)
    
    partition = partitionList[0]
    #Learning theta and partition
    if len(thetaList) >1 or len(partitionList)>1: #many choices 
        epsilon_theta = epsilon * 0.1
        epsilon_deg = epsilon-epsilon_theta
        (theta, partition) = learnThetaPartition(G, maxDeg, epsilon_theta, epsilon_deg, thetaList, partitionList)
        #print("sampled theta", theta, "sampled partition", partition)

    #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    degListGt = edgeAddition_DegList(G,theta)
    #print(degListGt)
    
    # historm + lap noise
    degHis = degSeqToDegHis(degListGt, theta)
    noisyDegHis = np.zeros(len(degHis))
    sens = 1 
    for i in range(len(partition)-1):
        start = partition[i]
        end = partition[i+1]
        if end >= (theta+1):
            end = min(end,theta+1)
        noisyAve = (sum(degHis[start:end]) + lap([0],sens,epsilon_deg)[0])/(end-start)
        for j in range(start,end):
            noisyDegHis[j] = noisyAve
        if end >= (theta+1):
            break 
                
    noisyDegHis = extendHis(noisyDegHis,maxDeg)
    #print("noisyCumDegHis - extend lengh", noisyDegHis[90:110])
    noisyDegHis = postTail(noisyDegHis, theta)
    #print("noisyDegHis - post tail", noisyDegHis[90:110])
    
    return noisyDegHis

In [35]:
#Caller 

dataDir ="Datasets/"
dataNames = ["HepPh.txt","toydata.txt", "facebook_combined.txt", "wiki-Vote.txt", 
             "email-Enron.txt",  "cit-HepTh.txt", "com-dblp.ungraph.txt"]
dataKey = 0
dataName = dataNames[dataKey]
print(dataName)

datafile = dataDir+dataName #"facebook_combined.txt"
G=nx.read_edgelist(datafile, nodetype=int)
#G = nx.fast_gnp_random_graph(20,0.2)

nodesNum = len(G.nodes()) #assume this is given
maxDeg = nodesNum -1  #assume this is given

trueHis = getDegHis(G,maxDeg)
print(trueHis)

nodesList = list(G.nodes)
print(min(nodesList),max(nodesList),len(nodesList))

HepPh.txt
[   0. 1492. 1802. ...    0.    0.    0.]
1 89208 12008


In [ ]:
##Testing of algorithms for degree distribution
epsilon = 10.0
maxTheta = min(nodesNum-1,200)
#noisyDegHis = nodeDP_nodeTrun_Smooth(G,maxDeg,epsilon, maxTheta)
###noisyDegHis = nodeDP_flowgraph_degSeq_Lap(G,maxDeg,epsilon, maxTheta) #works for small graph
#thetaCandidates = list(range(maxTheta))
thetaCandidates = [20,40,60,80,100,120,140,160,180,200]
#thetaCandidates = [100]#[nodesNum-1]
rCandidates = [1.2,1.5] #r determines the partition (g_i = {k:r^{i-1}<= k < r^i})
#noisyDegHis = nodeDP_edgeAdd_degHisPart_Lap(G, maxDeg, epsilon, thetaCandidates, rCandidates) 
#noisyDegHis = nodeDP_edgeAdd_degCum_Lap(G,maxDeg,epsilon,thetaCandidates)
noisyDegHis = nodeDP_edgeAdd_degCum_Lap_variant(G,maxDeg,epsilon,thetaCandidates)

#print("trueHist0-29", trueHis[0:30])
#print("noisyHist0-29", noisyDegHis[0:30])
plotHis(trueHis/nodesNum,noisyDegHis/nodesNum)
print(difDegHis_L1(trueHis/nodesNum, noisyDegHis/nodesNum))
print(difDegHis_L1(trueHis/nodesNum, (np.zeros(maxDeg+1))/nodesNum))
print(difDegHis_L1(trueHis, noisyDegHis))
print(difDegHis_L1(trueHis, (np.zeros(maxDeg+1))))

In [ ]:
cdf1=1-pdfToCdf(trueHis)
cdf2=1-pdfToCdf(noisyDegHis)

plotCum(cdf1,cdf2)

In [ ]:
###Evaluation of algorithms for degree distribution

algoNames = ["edgeDP_degHis_Lap", 
             "edgeDP_degSeq_Lap", 
             "nodeDP_degHis_Lap", 
             "nodeDP_degSeq_Lap", 
             "nodeDP_nodeTrun_Smooth", 
             "nodeDP_edgeAdd_degHisPart_Lap", 
             "nodeDP_edgeAdd_degCum_Lap",
             "nodeDP_edgeAdd_degCum_Lap_variant",
             "nodeDP_flowgraph_degSeq_Lap"]
algoKey = 6
algo = algoNames[algoKey]
print(algo)

#epsList = [0.01,0.02,0.05,0.1,0.2,0.5,1.0,2.0,5.0,10.0]
#epsList = [0.1, 0.5, 1.0, 1.5, 2.0]
epsList = [1.0]
repeats = 1

maxTheta = min(nodesNum-1,200)
thetaCandidates = [20,40,60,80,100,120,140,160,180,200]
if maxTheta < 200:
    thetaCandidates = [i+1 for i in range(maxTheta)]
#print(thetaCandidates)

for epsilon in epsList:
    errors = []
    for i in range(repeats):
        noisyDegHis = np.zeros(maxDeg+1)
        if algo == "edgeDP_degHis_Lap":        
            noisyDegHis = edgeDP_degHis_Lap(G,maxDeg,epsilon)
        elif algo == "edgeDP_degSeq_Lap":
            noisyDegHis = edgeDP_degSeq_Lap(G,maxDeg,epsilon)
        elif algo == "nodeDP_degHis_Lap":
            noisyDegHis = nodeDP_degHis_Lap(G,maxDeg,epsilon)
        elif algo == "nodeDP_degSeq_Lap":
            noisyDegHis = nodeDP_degSeq_Lap(G,maxDeg,epsilon)
        elif algo == "nodeDP_nodeTrun_Smooth":
            noisyDegHis = nodeDP_nodeTrun_Smooth(G,maxDeg,epsilon,maxTheta)
        elif algo == "nodeDP_edgeAdd_degHisPart_Lap":
            rCandidates = [1.2,1.5,1.8] #r determines the partition (g_i = {k:r^{i-1}<= k < r^i})
            noisyDegHis = nodeDP_edgeAdd_degHisPart_Lap(G, maxDeg, epsilon, thetaCandidates, rCandidates) 
        elif algo == "nodeDP_edgeAdd_degCum_Lap":
            noisyDegHis = nodeDP_edgeAdd_degCum_Lap(G,maxDeg,epsilon,thetaCandidates)
        elif algo == "nodeDP_edgeAdd_degCum_Lap_variant":
            noisyDegHis = nodeDP_edgeAdd_degCum_Lap_variant(G,maxDeg,epsilon,thetaCandidates)
        elif algo == "nodeDP_flowgraph_degSeq_Lap":
            noisyDegHis = nodeDP_flowgraph_degSeq_Lap(G,maxDeg,epsilon, maxTheta) #works for small graph
        else:
            print("no valid algo")
        noisyPdf = noisyDegHis/nodesNum
        #print(maxTheta)
        #print(trueHis)
        #print(noisyDegHis)
        error = difDegHis_L1(trueHis/nodesNum, noisyDegHis/nodesNum)
        #print(i, error)
        errors.append(error)
        #plotHis(trueHis/nodesNum,noisyDegHis/nodesNum)
        #plotCum(trueHis/nodesNum,noisyDegHis/nodesNum)
                
    print(epsilon, np.mean(errors), np.std(errors))



In [ ]:
#####unused code
##sparse version (my version, not useful for local sensitivity computation)

###full neighbors set (costly to compute and store)
start_time = time.time()
commonNeighbors = defaultdict(set)
for i in range(nodesNum):
    #print(set(G[i].keys()))
    for j in set(G[i].keys()):
        if j>i:
            commonN = set(G[i].keys()).intersection(set(G[j].keys()))
            #print(i,j, commonN)
            commonNeighbors.update({'{},{}'.format(i,j):commonN})

#print(commonNeighbors)
print("--- %s seconds ---" % (time.time() - start_time))



In [3]:
from scipy.special import comb
from collections import defaultdict
import time

class graphStat(object):
    #mainly store aggregated statistics of G
    
    def __init__(self, G):
        #take in networkx as an argument
    
        #degree number 
        self.nodesNum = len(G.nodes())

        #A_ij: the set of common neighbors of i and j 
        self.A = defaultdict(set)
        self.maxA = -1.0

        self.initSparseA(G)
         
    def initSparseA(self, G):
        start_time = time.time()
        for u,v in G.edges(): #edges in G only store one copy: either (u,v) or (v,u), not both
            for p in G[u]:
                if p != v:
                    self.A['{},{}'.format(min(p,v), max(p,v))].add(u)
            for p in G[v]:
                if p != u:
                    self.A['{},{}'.format(min(p,u),max(p,u))].add(v)
        print("--- %s seconds ---" % (time.time() - start_time))
        
        for commonNeighbors in self.A.values():
            self.maxA = max(self.maxA, len(commonNeighbors))
        
        
    def getA(self,i,j):
        return self.A['{},{}'.format(i,j)]
    
    
    def geta(self,i,j):
        return len(self.getA(i,j))
    



In [21]:
def count_clique(G,nodesRange,k):
    if(len(nodesRange)<k):
        return 0
    elif k==1:
        return len(nodesRange)

    count = 0
    for i in nodesRange:
        count += count_clique(G, set(G[i].keys()).intersection(nodesRange),k-1)
    return count/k

def count(G, Gstat, queryType, k):
    start_time = time.time()
    count = 0
    if queryType == "triangle":
        for u,v in G.edges():
            count = count + Gstat.geta(min(u,v),max(u,v))
        count = count/3 
    elif queryType == "kstar":
        for i in range(Gstat.nodesNum):
            d = G.degree[i]
            if d >= k:
                count = count + comb(d,k)
    elif queryType == "kclique":
        for u,v in G.edges():
            count += count_clique(G,Gstat.getA(min(u,v),max(u,v)), k-2)
        count /= comb(k,2)
    elif queryType == "ktriangle":
        for u,v in G.edges():
            count = count + comb(Gstat.geta(min(u,v),max(u,v)),k)    
    print("--- %s seconds ---" % (time.time() - start_time))
    return count 

In [22]:
##Edge-DP 
def edgeDP_computeGS(G, Gsstat, queryType, k):
    nodesNum = Gstat.nodesNum
    if queryType == "triangle":
        return nodesNum-2
    elif queryType == "kstar":
        return 2 * comb(nodesNum-2, k-1)
    elif queryType == "kclique":
        return comb(nodesNum-2,k-2)
    elif queryType == "ktriangle":
        return comb(nodesNum-2,k) + 2*(nodesNum-2)*comb(nodesNum-3,k-1)
    else:
        print(queryType, "is unspecified")
        return -1.0

In [6]:
import os.path

#Caller 

dataDir ="Datasets/"
dataNames = ["HepPh","toydata", "facebook_combined", "wiki-Vote", 
             "email-Enron",  "cit-HepTh", "com-dblp.ungraph"]
dataKey = 2
dataName = dataNames[dataKey]

datafile = dataDir+dataName #"facebook_combined.txt"
if not os.path.isfile(datafile+"-map.txt"):
    translate(datafile+".txt", datafile+"-map.txt")
    #convert all nodes id to 0 to nodeNum
else:
    print("file exists")
    
datafile = datafile + "-map.txt"
print(datafile)

G=nx.read_edgelist(datafile, nodetype=int)
#G = nx.fast_gnp_random_graph(20,0.2)
G.remove_edges_from(nx.selfloop_edges(G))
print(len(G.edges))

nodesNum = len(G.nodes()) #assume this is given
maxDeg = nodesNum -1  #assume this is given

trueHis = getDegHis(G,maxDeg)
print(trueHis)

nodesList = list(G.nodes)
print(min(nodesList),max(nodesList),len(nodesList))

print(len(G.edges))



file exists
Datasets/facebook_combined-map.txt
88234
[ 0. 75. 98. ...  0.  0.  0.]
0 4038 4039
88234


In [7]:
Gstat = graphStat(G)

--- 25.199180126190186 seconds ---


In [23]:
####triangle 
print(count(G,Gstat,"triangle",-1))

####k-star, k=3
print(count(G,Gstat,"kstar",3))

####k-clique, k=4 
print(count(G,Gstat,"kclique",4))

####k-triangle, k=2 
print(count(G,Gstat,"ktriangle",2))


--- 0.18755507469177246 seconds ---
1612010.0
--- 0.035372018814086914 seconds ---
727318426.0
--- 86.82794713973999 seconds ---
30004668.0
--- 0.6820480823516846 seconds ---
228787050.0


In [8]:
def edgeDP_computeLS(G, Gstat, queryType, k):
    ls = 0.0 
    if queryType == "triangle":
        ls = Gstat.maxA
        return ls
    elif queryType == "kstar":
        #TODO: incomplete
        return ls
    elif queryType == "kclique":
        #TODO: incomplete
        return ls
    elif queryType == "ktriangle":
        #TODO: incomplete
        return ls
    else:
        print(queryType, "is unspecified")
        return ls

In [9]:
def edgeDP_LadderFunction(G, Gstat, queryType, k):
    lsd = []    
    if queryType == "triangle":
        lsd = lsd_triangle(G,Gstat,queryType,k)
    elif queryType == "kstar":
         #TODO: incomplete
        return lsd
    elif queryType == "kclique":
         #TODO: incomplete
        return lsd
    elif queryType == "ktriangle":
        #TODO: incomplete
        return lsd
    else:
        print(queryType, "is unspecified")
    return lsd



In [26]:
def lsd_triangle(G,Gstat,queryType,k):
    start_time = time.time()
    nodesNum = Gstat.nodesNum
    bucket = [-1] * nodesNum #bucket: the common neighbor sizes 
    for i in range(nodesNum):
        for j in range(i+1, nodesNum):
            #aij: the number of common neighbors of i and j
            aij = Gstat.geta(i,j)
            #di = G.degree[i]
            #dj = G.degree[j]
            #xij = int(G.has_edge(i,j))
            #bij: the number of nodes connected to exactly one of i and j
            bij = G.degree[i] + G.degree[j] - 2*aij - 2*int(G.has_edge(i,j))
            bucket[aij] = max(bucket[aij], bij)  
            #if (i==0 and j <10):
            #    print(i,j,aij, bucket[aij], di,dj,xij)
            
    print("bucket(0-10):", bucket[0:10])
    
    uppers = []
    for i in reversed(range(nodesNum)):
        if bucket[i] <0:
            continue
        if (len(uppers)==0) or (i*2+bucket[i] > uppers[-1][0]*2 + uppers[-1][1]):
            uppers.append([i, bucket[i]])
    #print("uppers(0-10):", uppers[0:10])
    
    gs = edgeDP_computeGS(G,Gstat,queryType,k)
    #print("gs: ", gs)
    
    LSD = []
    t = 0
    
    while 1:
        lsd = 0
        for p in uppers:
            lsd = max(lsd, p[0]+ (t+min(t,p[1])) /2)
        t +=1 
        if lsd < gs:
            LSD.append(lsd)
        else:
            LSD.append(gs)
            print("LSD(0-10):", LSD[0:10])
            print("--- %s seconds ---" % (time.time() - start_time))
            return LSD    
          

In [17]:
####triangle 
queryType = "triangle"
k=-1
#print(edgeDP_computeLS(G,Gstat,queryType,k))
lsd = edgeDP_LadderFunction(G, Gstat, queryType, k)

0 1 16 330 347 17 1
0 2 9 337 347 10 1
0 3 16 330 347 17 1
0 4 9 337 347 10 1
0 5 12 334 347 13 1
0 6 5 341 347 6 1
0 7 19 327 347 20 1
0 8 7 339 347 8 1
0 9 56 290 347 57 1
bucket(0-10): [1339, 1590, 1386, 1333, 1077, 1095, 1788, 1079, 1090, 1093]
uppers(0-10): [[293, 461], [253, 791], [14, 1807]]
gs:  4037
LSD(0-10): [293.0, 294.0, 295.0, 296.0, 297.0, 298.0, 299.0, 300.0, 301.0, 302.0]
--- 86.31673192977905 seconds ---


In [44]:
#ladder function paper: algorithm 1 
def edgeDP_LadderMechanism(G, Gstat, queryType, k, epsilon, repetition):
    trueCount = count(G,Gstat,queryType,k)
    print(queryType,k, "true count: ", trueCount)
    
    #ladders: ladder function evaluated on G
    ladders = edgeDP_LadderFunction(G,Gstat,queryType,k)  
    #M: length of the ladder function
    M = len(ladders)
    
    ranges = []
    weights = [1.0] #the center's weight
    
    #rungs 1 to M
    dst = 0.0 
    for t in range(M):
        weights.append(2*ladders[t]*np.exp(epsilon/2.0*(-t-1)))
        ranges.append(dst)
        dst = dst + ladders[t]
        
    #rung M+1
    weights.append(2*ladders[-1]* np.exp(epsilon/2.0*(-M-1))/(1-np.exp(-epsilon/2.0)))
        
    ####the only part that involves randomness, may store the earlier results for evaluation over multiple runs 
    
    errors = []
    for i in range(repetition):
        noisyCount = 0
        
        #run exponential mechanism over the weights
        t = int(sampleProbList(weights))

        if t==0:
            noisyCount = trueCount

        elif t<= M:
            flag = -1.0
            if (np.random.uniform()>0.5):
                flag = 1.0
            low = ranges[t-1]
            delta = np.ceil(np.random.uniform() * (ranges[t] - ranges[t-1]))
            noisyCount = flag * delta + trueCount

        else:
            p = 1.0 - np.exp(-epsilon/2.0)
            ext = np.random.geometric(p)
            low = dst + ext * ladders[-1]
            high = low + ladders[-1]
            flag = -1.0
            if (np.random.uniform()>0.5):
                flag = 1.0
            noisyCount = flag * np.random.randint(low, high+1) + trueCount
        
        errors.append(abs(noisyCount-trueCount)/trueCount)
    return errors
        
    

In [45]:
queryType = "triangle"
k=-1
epsilon = 0.05
repetition = 30
errors = edgeDP_LadderMechanism(G, Gstat, queryType, k, epsilon,repetition)
print(np.mean(errors), np.std(errors))

--- 0.23247575759887695 seconds ---
triangle -1 true count:  1612010.0
bucket(0-10): [1339, 1590, 1386, 1333, 1077, 1095, 1788, 1079, 1090, 1093]
uppers(0-10): [[293, 461], [253, 791], [14, 1807]]
gs:  4037
LSD(0-10): [293.0, 294.0, 295.0, 296.0, 297.0, 298.0, 299.0, 300.0, 301.0, 302.0]
--- 57.03488898277283 seconds ---
9.367187548464339e-05


In [50]:
queryType = "triangle"
k=-1
epsilon = 1.6
repetition = 100

if 1:
    trueCount = count(G,Gstat,queryType,k)
    print(queryType,k, "true count: ", trueCount)
    
    #ladders: ladder function evaluated on G
    ladders = lsd
    #M: length of the ladder function
    M = len(ladders)
    
    ranges = []
    weights = [1.0] #the center's weight
    
    #rungs 1 to M
    dst = 0.0 
    for t in range(M):
        weights.append(2*ladders[t]*np.exp(epsilon/2.0*(-t-1)))
        ranges.append(dst)
        dst = dst + ladders[t]
        
    #rung M+1
    weights.append(2*ladders[-1]* np.exp(epsilon/2.0*(-M-1))/(1-np.exp(-epsilon/2.0)))
    
    print(sum(weights))
    
    ####the only part that involves randomness, may store the earlier results for evaluation

    errors = []
    for i in range(repetition):
        noisyCount = 0
        
        #run exponential mechanism over the weights
        t = int(sampleProbList(weights))
        
        if t==0:
            noisyCount = trueCount

        elif t<= M:
            flag = -1.0
            if (np.random.uniform()>0.5):
                flag = 1.0
            low = ranges[t-1]
            delta = low + np.ceil(np.random.uniform() * (ranges[t] - ranges[t-1]))
            noisyCount = flag * delta + trueCount

        else:
            p = 1.0 - np.exp(-epsilon/2.0)
            ext = np.random.geometric(p)
            low = dst + ext * ladders[-1]
            high = low + ladders[-1]
            flag = -1.0
            if (np.random.uniform()>0.5):
                flag = 1.0
            noisyCount = flag * np.random.randint(low, high+1) + trueCount
        
        errors.append(abs(noisyCount-trueCount)/trueCount)
        
    print(np.median(errors))
    print(np.mean(errors), np.std(errors))

--- 0.16189217567443848 seconds ---
triangle -1 true count:  1612010.0
480.48780720418335
0.0001575672607490028
0.00024371436901756194 0.00026513878905179395
